# Matrix Network

## Imports

In [ ]:
# Photontorch
import photontorch as pt

# Python
import tqdm
import torch
import numpy as np
import matplotlib.pyplot as plt

# numpy settings
np.random.seed(6) # seed for random numbers
np.set_printoptions(precision=2, suppress=True) # show less numbers while printing numpy arrays

## Schematic
![Unitary Matrix Paper](images/clements.jpeg)

(a) Reck Design

(b) Clements Design

## Simulation and Design Parameters

Here we will use the matrix network *with* delays.

In [ ]:
length = 25e-6 #[m]
transmission = 0.5 #[]
neff = 2.86
env = pt.Environment(
    t_start = 0,
    t_end = 2000e-14,
    dt = 1e-14,
    wl = 1.55e-6,
)
pt.set_environment(env)

## (a) Reck Design

In [ ]:
nw = pt.ReckMxN(M=4, N=2, length=1e-4).terminate()
source = np.ones(2)/np.sqrt(2) # 2 sources -> Make total input power equal to 1

### Simulation

In [ ]:
detected_time = nw(source)
nw.plot(detected_time[:,0,:,0]); # plot first and only batch

### Total power recovered:

In [ ]:
detected_time[-1,0,:,0].sum()

### Optimizing the coupling

The goal is to optimize the coupling of the network such that we have the same output at the 4 detectors with an as high as possible amplitude (ideally, higher than in the equal coupling case).

In [ ]:
def train_for_same_output(nw, num_epochs=50, learning_rate=0.1):
    # Running this cell takes quite long
    target = torch.tensor([0.25,0.25,0.25,0.25], device=nw.device)
    lossfunc = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(nw.parameters(), lr=learning_rate)
    with pt.Environment(wl=1.55e-6, t_start=0, t_end=5000e-15, dt=25e-15, enable_grad=True):
        for epoch in tqdm.trange(num_epochs):
            det_train = nw(source)[-1,0,:,0] # get first and only batch
            loss = lossfunc(det_train, target)
            loss.backward()
            optimizer.step()
            del det_train, loss # Free up memory (mostly important when working on GPU)
        
train_for_same_output(nw)

### Final Simulation

In [ ]:
%time det_train = nw(source)
nw.plot(det_train[:,0,:,0]); # plot first and only batch

Note that in the Reck network, signals arrive at different times.

### Total power recovered

In [ ]:
det_train[-1,0,:,0].sum()

## (b) Clements Design

In [ ]:
nw = pt.ClementsNxN(N=4, length=1e-4).terminate(term=[
    pt.Source('s0'),
    pt.Source('s1'),
    pt.Term('t0'),
    pt.Term('t1'),
    pt.Detector('d0'),
    pt.Detector('d1'),
    pt.Detector('d2'),
    pt.Detector('d3'),
])
source = np.ones(2)/np.sqrt(2) # 2 sources -> Make total input power equal to 1

### Simulation

In [ ]:
detected_time = nw(source)
nw.plot(detected_time[:,0,:,0]); # plot first and only batch

### Total power recovered:

In [ ]:
detected_time[-1,0,:,0].sum()

### Optimizing the coupling

In [ ]:
def train_for_same_output(nw, num_epochs=50, learning_rate=0.1):
    # Running this cell takes quite long
    target = torch.tensor([0.25,0.25,0.25,0.25], device=nw.device)
    lossfunc = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(nw.parameters(), lr=learning_rate)
    with pt.Environment(wl=1.55e-6, t_start=0, t_end=5000e-15, dt=25e-15, enable_grad=True):
        for epoch in tqdm.trange(num_epochs):
            det_train = nw(source)[-1,0,:,0] # get first and only batch
            loss = lossfunc(det_train, target)
            loss.backward()
            optimizer.step()
            del det_train, loss # Free up memory (mostly important when working on GPU)
        
train_for_same_output(nw)

### Final Simulation

In [ ]:
%time det_train = nw(source)
nw.plot(det_train[:,0,:,0]); # plot first and only batch

Note that in the Clements network, all signals arrive at the same time at the detector.